<a href="https://colab.research.google.com/github/mpalominocol/Projects/blob/main/WB_Data_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
# Storage information in a data frame with country, year, quarter and indicators, each indicators as a column indicators
import pandas as pd
import requests
import json

countries = ['US','CA','FR','DE','IT','JP','GB','US','CN','RU']
indicators = ['SP.POP.TOTL', 'NY.GDP.MKTP.CD', 'EG.USE.EG.KO.PT', 'EN.ATM.CO2E.KT', 'SH.STA.MMRT', 'SE.PRM.PRSL', 'EG.ELC.ACCS.ZS', 'SL.AGR.EMPL.ZS', 'EN.ATM.FOREST.K2', 'AG.LND.AGRI.ZS', 'FP.CPI.TOTL']
start_date = '2019Q1'
end_date = '2024Q4'

data = []

for country in countries:
    for indicator in indicators:
        url = f'https://api.worldbank.org/v2/country/{country}/indicator/{indicator}?date={start_date}:{end_date}&frequency=Q&format=json'
        response = requests.get(url)
        try:
            data_json = response.json()
            if len(data_json) > 1:  # Check if data is available
                for item in data_json[1]:
                    data.append([
                        country,
                        item['date'][:4], #Year
                        item['date'][4:], #Quarter
                        indicator,
                        item['value']
                    ])
        except (json.JSONDecodeError, KeyError, IndexError) as e:
            print(f"Error processing data for {country}, {indicator}: {e}")
            print(response.text)

df = pd.DataFrame(data, columns=['Country', 'Year', 'Quarter', 'Indicator', 'Value'])

# Pivot the DataFrame to have indicators as columns
df = df.pivot_table(index=['Country', 'Year', 'Quarter'], columns='Indicator', values='Value').reset_index()
df

Indicator,Country,Year,Quarter,AG.LND.AGRI.ZS,EG.ELC.ACCS.ZS,FP.CPI.TOTL,NY.GDP.MKTP.CD,SH.STA.MMRT,SL.AGR.EMPL.ZS,SP.POP.TOTL
0,CA,1974,,7.038466,NaN,22.459931,1.609492e+11,NaN,NaN,22807969.0
1,CA,1975,,7.041477,NaN,24.856898,1.744198e+11,NaN,NaN,23143275.0
2,CA,1976,,7.044599,NaN,26.731540,2.072716e+11,NaN,NaN,23449808.0
3,CA,1977,,7.002885,NaN,28.863766,2.123252e+11,NaN,NaN,23725843.0
4,CA,1978,,6.961059,NaN,31.453921,2.193695e+11,NaN,NaN,23963203.0
...,...,...,...,...,...,...,...,...,...,...
445,US,2019,,44.363367,100.0,117.244195,2.152140e+13,20.0,1.679713,328329953.0
446,US,2020,,44.363367,100.0,118.690502,2.132295e+13,21.0,1.746393,331526933.0
447,US,2021,,44.363367,100.0,124.266414,2.359403e+13,NaN,1.662403,332048977.0
448,US,2022,,NaN,100.0,134.211206,2.574411e+13,NaN,1.620543,333271411.0


In [25]:
# Export to CSV
df.to_csv('world_bank_data.csv', index=False)

In [26]:
# Cross table of country and year

pd.crosstab(df['Country'], df['Year'])


Year,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Country,,,,,,,,,,,,,,,,,,,,,
CA,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
CN,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
DE,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
FR,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
GB,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
IT,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
JP,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
RU,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
US,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
